In [3]:
import os
import numpy as np
from models.att_models import att_baseline
from keras.callbacks import ModelCheckpoint, EarlyStopping
from data_utility.file_utility import FileUtility
from data_utility.labeling_utility import LabelingData
import itertools
from data_utility.feedgenerator import train_batch_generator_408, validation_batch_generator_408
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [4]:
def run_baseline(epochs=10, setting_name='basemodel', gpu='1', train_batch_size=64,
                 test_batch_size=100, patience=10,lstm_size =1000, features_to_use=['onehot', 'sequence_profile'], convs=[3, 5, 7], dense_size=200,use_CRF=False,filter_size=256):
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu

    # read files
    train_file = '../DeepSeq2Sec/data/s8_all_features/train.txt'
    test_file = '../DeepSeq2Sec/data/s8_all_features/test.txt'
    LD = LabelingData(train_file, test_file)
    train_lengths = [int(j) for j in FileUtility.load_list('/'.join(train_file.split('/')[0:-1]) + '/train_length.txt')]
    test_lengths = [int(i) for i in FileUtility.load_list('/'.join(test_file.split('/')[0:-1]) + '/test_length.txt')]

    # model
    model, params = att_baseline(LD.n_classes, features_to_use=features_to_use, convs=convs,
                             dense_size=dense_size, use_CRF=use_CRF,lstm_size =lstm_size, filter_size=filter_size)

    # output directory
    FileUtility.ensure_dir('results_att/' + setting_name + params + '/')

    # save model
    with open('results_att/' + setting_name + params + '/' + 'config.txt', 'w') as fh:
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    # check points
    filepath = 'results_att/' + setting_name + params + "/weights-improvement-{epoch:02d}-{weighted_acc:.3f}-{val_weighted_acc:.3f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_weighted_acc', verbose=1, save_best_only=True, mode='max',
                                 period=1)
    earlystopping = EarlyStopping(monitor='val_weighted_acc', min_delta=0, patience=patience, verbose=0, mode='max',
                                  baseline=None)
    callbacks_list = [checkpoint, earlystopping]

    # calculate the sizes
    steps_per_epoch = len(train_lengths) / train_batch_size if len(train_lengths) % train_batch_size == 0 else int(
        len(train_lengths) / train_batch_size) + 1
    validation_steps = int(len(test_lengths) / test_batch_size) if len(test_lengths) % test_batch_size == 0 else int(
        len(test_lengths) / test_batch_size) + 1

    # feed model
    h = model.fit_generator(train_batch_generator_408(train_batch_size), steps_per_epoch=steps_per_epoch,
                            validation_data=validation_batch_generator_408(test_batch_size),
                            validation_steps=validation_steps,
                            shuffle=False, epochs=epochs, verbose=1, callbacks=callbacks_list)

    # save the history
    FileUtility.save_obj('results_att/' + setting_name + params + '/history', h.history)
    return model




In [5]:
model=run_baseline(epochs=1000, setting_name='attention_additive_', gpu='0', train_batch_size=64,
                 test_batch_size=64, patience=50, features_to_use=['onehot', 'sequence_profile'], convs=[3,5,7,11,21], lstm_size=1000, dense_size=1000, use_CRF=False, filter_size=256)

Labeling utility object created..
Training y encoded shape is  (5534, 700)
Maximum sequence length is 700
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 408)    0                                            
__________________________________________________________________________________________________
onehot (Lambda)                 (None, None, 21)     0           input_1[0][0]                    
__________________________________________________________________________________________________
sequenceprofile (Lambda)        (None, None, 21)     0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 42)     0           onehot[0][0]                     
   

Epoch 8/1000
87/87 [==============================] - 27s 310ms/step - loss: 0.7493 - weighted_acc: 0.7539 - val_loss: 0.9699 - val_weighted_acc: 0.6823

Epoch 00008: val_weighted_acc improved from 0.67741 to 0.68232, saving model to results_att/attention_additive_model#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_256/weights-improvement-08-0.754-0.682.hdf5
Epoch 9/1000
87/87 [==============================] - 27s 308ms/step - loss: 0.7511 - weighted_acc: 0.7553 - val_loss: 0.9678 - val_weighted_acc: 0.6840

Epoch 00009: val_weighted_acc improved from 0.68232 to 0.68403, saving model to results_att/attention_additive_model#onehot#sequence_profile@conv3_5_7_11_21@dense_1000@lstm1000@droplstm0.5@filtersize_256/weights-improvement-09-0.756-0.684.hdf5
Epoch 10/1000
87/87 [==============================] - 26s 303ms/step - loss: 0.7213 - weighted_acc: 0.7664 - val_loss: 0.9614 - val_weighted_acc: 0.6846

Epoch 00010: val_weighted_acc improved from 0.684

87/87 [==============================] - 27s 307ms/step - loss: 0.2232 - weighted_acc: 0.9445 - val_loss: 1.5101 - val_weighted_acc: 0.6598

Epoch 00042: val_weighted_acc did not improve from 0.68501
Epoch 43/1000
87/87 [==============================] - 27s 310ms/step - loss: 0.2132 - weighted_acc: 0.9482 - val_loss: 1.5388 - val_weighted_acc: 0.6575

Epoch 00043: val_weighted_acc did not improve from 0.68501
Epoch 44/1000
87/87 [==============================] - 27s 311ms/step - loss: 0.2127 - weighted_acc: 0.9488 - val_loss: 1.5442 - val_weighted_acc: 0.6554

Epoch 00044: val_weighted_acc did not improve from 0.68501
Epoch 45/1000
87/87 [==============================] - 27s 312ms/step - loss: 0.2080 - weighted_acc: 0.9506 - val_loss: 1.6609 - val_weighted_acc: 0.6483

Epoch 00045: val_weighted_acc did not improve from 0.68501
Epoch 46/1000
87/87 [==============================] - 27s 308ms/step - loss: 0.2047 - weighted_acc: 0.9520 - val_loss: 1.6144 - val_weighted_acc: 0.6581

Epo